In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from catboost import CatBoostClassifier
from xgboost import XGBClassifier


In [3]:
df = pd.read_csv('A:\ML-Project\SaftoStep\data\delhi_crime_data.csv')
df.head()

,Crime Type,Area,Date,Time,Severity,Crime Description,Victim Age,Police Station,Weather Condition,Time of Day,Incident Reported
0,Assault,Khan Market,2023-09-27,00:33:02,Medium,Bad door stand detail own strong decision.,39,Khan Market PS,Clear,Night,CCTV
1,Stalking,Connaught Place,2024-04-17,11:16:22,Low,Between instead author lawyer born.,62,Connaught Place PS,Foggy,Afternoon,Police
2,Stalking,Mayur Vihar,2024-12-25,04:51:08,High,Loss sound enough answer paper nor decision.,63,Mayur Vihar PS,Cloudy,Evening,Police
3,Kidnapping,Laxmi Nagar,2024-03-13,09:52:32,High,Ago edge draw yard impact behavior political h...,66,Laxmi Nagar PS,Cloudy,Morning,Citizen
4,Theft,Hauz Khas,2024-08-05,07:30:49,High,Evidence development specific enough standard ...,34,Hauz Khas PS,Foggy,Evening,CCTV


In [4]:
df.isnull().sum()
df = df.drop(columns=["Crime Description","Police Station","Time","Date"])
df.head()

,Crime Type,Area,Severity,Victim Age,Weather Condition,Time of Day,Incident Reported
0,Assault,Khan Market,Medium,39,Clear,Night,CCTV
1,Stalking,Connaught Place,Low,62,Foggy,Afternoon,Police
2,Stalking,Mayur Vihar,High,63,Cloudy,Evening,Police
3,Kidnapping,Laxmi Nagar,High,66,Cloudy,Morning,Citizen
4,Theft,Hauz Khas,High,34,Foggy,Evening,CCTV


In [5]:
#encode categorical data
label_encoders = {}
categorical_cols = ["Crime Type","Area","Weather Condition", "Time of Day", "Incident Reported"]
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

In [6]:
#independent and dependent variables
X = df.drop(columns=["Severity"])
y = df["Severity"]

In [7]:
# Convert severity to binary (Safe = 0, Risky = 1)
y = y.map(lambda x: 0 if x== "Low" else 1)
y.unique()

array([1, 0], dtype=int64)

In [8]:
#standardize the model
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Support Vector Machine": SVC(kernel='linear', probability=True),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5)
}

best_model = None
best_score = 0
for name, model in models.items():
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    #print(f"{name} Accuracy: {accuracy:.4f}")
    #print(classification_report(y_test, y_pred))

    cv_score = np.mean(cross_val_score(model, X, y, cv=5))
    #print(f"{name} Cross-Validation Score: {cv_score:.4f}\n")

    if cv_score > best_score:
        best_score = cv_score
        best_model = model


In [13]:
print(f"{best_model}")
print(f"{best_score:.4f}")

LogisticRegression(max_iter=1000)
0.6664



def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

models = {
    "Logistic Regression": LogisticRegression(),
    "K-Neighbors Classifier": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest Classifier": RandomForestClassifier(),
    "XGBoost Classifier": XGBClassifier(),
    "CatBoost Classifier": CatBoostClassifier(verbose=False),
    "AdaBoost Classifier": AdaBoostClassifier()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.2f}%".format(model_train_r2 * 100))

    print("------------------------------------------------------")

    print('Model Performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.2f}%".format(model_test_r2 * 100))
    r2_list.append(model_test_r2)

    print("=" * 34)
    print('\n')


print(pd.DataFrame(list(zip(model_list, r2_list)),columns=['Model Name', 'R2 Score']).sort_values(by=['R2 Score'],ascending=False))
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
